In [1]:
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json, requests

In [4]:
class PrizePicks():
    def __init__(self):
        self.url = 'view-source:https://api.prizepicks.com/projections?league_id=7'
        self.options = FirefoxOptions()
        self.options.add_argument('--headless')
        self.driver = webdriver.Firefox(options=self.options)
    def fetchProps(self, read_from_file = False):
        if read_from_file:
            json_data = json.load(open('../data/projections.json'))
        else: 
            self.driver.get(self.url)
            data = self.driver.find_element(By.TAG_NAME, 'pre').text
            json_data = json.loads(data)
            with open('./data/projections.json', 'w') as out:
                out.write(json.dumps(json_data))
            
        seive = {"points", "rebounds", "assists", "threes", "blocks", "steals", "pra", "pr", "pa", "ra"}
        player_names = {elem["id"]: elem["attributes"]["name"]
                        for elem in json_data["included"]
                        if elem["type"] == "new_player"}
        player_projections = []
        for projection in json_data["data"]:
            if projection["type"] == "projection":
                player_id = projection["relationships"]["new_player"]["data"]["id"]
                player_name = player_names.get(player_id, "Unknown Player")
                projection_id = projection['id']

                flash_sale = projection["attributes"].get("flash_sale_line_score")
                line_score = projection["attributes"]["line_score"]
                player_team = projection["attributes"]["description"]
                stat_type = self.statType(projection["attributes"]["stat_type"]).lower()
                start_time = projection["attributes"]["start_time"]

                if stat_type in seive and projection["attributes"].get("adjusted_odds") is not True:
                    player_projections.append({
                        'projection_id': projection_id,
                        'player_name': player_name,
                        'player_team' : player_team,
                        'stat_type': stat_type,
                        'line_score': line_score,
                        'start_time': start_time
                    })

                if stat_type in seive and flash_sale is not None:
                    player_projections.append({
                        'projection_id': projection_id,
                        'player_name': player_name,
                        'player_team' : player_team,
                        'stat_type': stat_type,
                        'line_score': flash_sale,
                        'start_time': start_time
                    })

        return player_projections
    def statType(self, stat):
        # find the stat retrieved
            if stat == "Pts+Rebs+Asts":
                return "pra"
            elif stat == "Pts+Asts":
                return "pa"
            elif stat == "Pts+Rebs":
                return "pr"
            elif stat == "Rebs+Asts":
                return "ra"
            elif stat == "3-PT Made":
                return "threes"
            elif stat == "Blocked Shots":
                return "blocks"
            else:
                return stat


In [8]:
import pandas as pd
from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from nba_api.stats.endpoints import PlayerGameLog, ScoreboardV2, CommonTeamRoster
from nba_api.stats.static import players



In [9]:
class PropsModelV1():
    def __init__(self, player_name, stat_type, line_score):
        self.player_name = player_name
        self.stat_type = stat_type
        self.line_score = line_score

    def predict(self):
        player_id = players.find_players_by_full_name(self.player_name)[0]["id"]
        print(players.find_players_by_full_name(self.player_name)[0])

        game_log = PlayerGameLog(player_id=player_id, season="2023-24").get_data_frames()[0]
        game_log2 = PlayerGameLog(player_id=player_id, season="2022-23").get_data_frames()[0]
        column = game_log[['Game_ID', 'GAME_DATE', 'MATCHUP', 'PTS', 'AST', 'STL', 'REB', 'TOV','FGA','FGM', 'FG3M', 'FG3A', 'BLK','FTA','FTM','PLUS_MINUS']]
        column2 = game_log2[['Game_ID', 'GAME_DATE', 'MATCHUP', 'PTS', 'AST', 'STL', 'REB', 'TOV','FGA','FGM', 'FG3M', 'FG3A', 'BLK','FTA','FTM','PLUS_MINUS']]
        frames = [pd.DataFrame(column), pd.DataFrame(column2)]
        df = pd.concat(frames)

        #df['PTS+REB+AST'] = df['PTS'] + df['REB'] + df['AST']
        #df['PTS+AST'] = df['PTS'] + df['AST']
        #df['REB+AST'] = df['REB'] + df['AST']
        #df['PTS+REB'] = df['PTS'] + df['REB']
        #df['BLKS+STLS'] = df['BLK'] + df['STL']

        df['HOME'] = df['MATCHUP'].apply(lambda x: 1 if 'vs. ' in x else 0)
        df['OPPONENT'] = df['MATCHUP'].apply(lambda x: x.split()[-1])

        if self.stat_type == 'points':
            features = ['HOME', 'AST', 'STL', 'REB', 'TOV', 'FG3M', 'FG3A', 'BLK', 'FGA', 'FGM', 'FTA', 'FTM', 'PLUS_MINUS']
            target = 'PTS'
        elif self.stat_type == 'rebounds':
            features = ['HOME', 'AST', 'STL', 'PTS', 'TOV', 'FG3M', 'FG3A', 'BLK']
            target = 'REB'
        elif self.stat_type == 'assists':
            features = ['HOME', 'PTS', 'STL', 'REB', 'TOV', 'FG3M', 'FG3A', 'BLK']
            target = 'AST'

        x = df[features]
        y = df[target]

        y = y.apply(lambda x: 1 if x > self.line_score else 0)

        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)

        x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.4, random_state=42)

        logistic_regression_model = LogisticRegression(max_iter=1000)
        logistic_regression_model.fit(x_train, y_train)

        predictions = logistic_regression_model.predict(x_test)

        accuracy = accuracy_score(y_test, predictions)
        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)
        f1 = f1_score(y_test, predictions)

        last3games = df[0:3]
        d3 = {}
        for f in features:
            d3[f] = last3games[f].mean()
        todays_games = pd.DataFrame([d3])
        todays_games_scaled = scaler.transform(todays_games)
        predictions2 = logistic_regression_model.predict(todays_games_scaled)

        return {
            'player_id': player_id,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'prediction': 'OVER' if predictions2[0] == 1 else 'UNDER'
        }


In [10]:
prize_picks = PrizePicks()
player_projections = prize_picks.fetchProps(read_from_file=True)
data = []
supported_stats = ['points', 'assists', 'rebounds']
for idx, player in enumerate(player_projections):

    if(player['stat_type'] in supported_stats):
        print(f"PREDICTING {player['player_name']} AT {player['line_score']} {player['stat_type']} ({idx}/{len(player_projections)})")
        try:
            model = PropsModelV1(player['player_name'], player['stat_type'], player['line_score'])
            prediction = model.predict()
            player.update(prediction)
            print(player)
            data.append(player)
            time.sleep(.5)
        except Exception as e:
            print(e)
            print(f"ERROR OCCURED WHILE PREDICTING {player['player_name']} AT {player['line_score']} {player['stat_type']}")

# with open('./data/prizepicks_predicted_props.json', 'w') as out:
#     out.write(json.dumps({'props': data, 'date': str(date.today())}))

data

PREDICTING Joel Embiid AT 31.5 points (1/460)
{'id': 203954, 'full_name': 'Joel Embiid', 'first_name': 'Joel', 'last_name': 'Embiid', 'is_active': True}
{'projection_id': '2273510', 'player_name': 'Joel Embiid', 'player_team': 'ORL', 'stat_type': 'points', 'line_score': 31.5, 'start_time': '2024-04-12T19:10:00-04:00', 'player_id': 203954, 'accuracy': 0.9523809523809523, 'precision': 0.9629629629629629, 'recall': 0.9629629629629629, 'f1': 0.9629629629629629, 'prediction': 'OVER'}
PREDICTING Joel Embiid AT 10.5 rebounds (2/460)
{'id': 203954, 'full_name': 'Joel Embiid', 'first_name': 'Joel', 'last_name': 'Embiid', 'is_active': True}
{'projection_id': '2273529', 'player_name': 'Joel Embiid', 'player_team': 'ORL', 'stat_type': 'rebounds', 'line_score': 10.5, 'start_time': '2024-04-12T19:10:00-04:00', 'player_id': 203954, 'accuracy': 0.5952380952380952, 'precision': 0.6, 'recall': 0.6818181818181818, 'f1': 0.6382978723404256, 'prediction': 'OVER'}
PREDICTING Franz Wagner AT 17.5 points (3/4

c:\Users\111356.TISD-AD\Desktop\props-ai\props.ai\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'projection_id': '2275699', 'player_name': 'Paolo Banchero', 'player_team': 'PHI', 'stat_type': 'assists', 'line_score': 5.5, 'start_time': '2024-04-12T19:10:00-04:00', 'player_id': 1631094, 'accuracy': 0.639344262295082, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'prediction': 'UNDER'}
PREDICTING Paolo Banchero AT 22.5 points (16/460)
{'id': 1631094, 'full_name': 'Paolo Banchero', 'first_name': 'Paolo', 'last_name': 'Banchero', 'is_active': True}
{'projection_id': '2275673', 'player_name': 'Paolo Banchero', 'player_team': 'PHI', 'stat_type': 'points', 'line_score': 22.5, 'start_time': '2024-04-12T19:10:00-04:00', 'player_id': 1631094, 'accuracy': 0.9016393442622951, 'precision': 1.0, 'recall': 0.7777777777777778, 'f1': 0.875, 'prediction': 'UNDER'}
PREDICTING Brandon Miller AT 5 rebounds (24/460)
{'id': 1641706, 'full_name': 'Brandon Miller', 'first_name': 'Brandon', 'last_name': 'Miller', 'is_active': True}
{'projection_id': '2275468', 'player_name': 'Brandon Miller', 'player_team'

c:\Users\111356.TISD-AD\Desktop\props-ai\props.ai\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'projection_id': '2275473', 'player_name': 'Tre Mann', 'player_team': 'BOS', 'stat_type': 'assists', 'line_score': 5.5, 'start_time': '2024-04-12T19:40:00-04:00', 'player_id': 1630544, 'accuracy': 0.8409090909090909, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'prediction': 'UNDER'}
PREDICTING Tre Mann AT 12.5 points (32/460)
{'id': 1630544, 'full_name': 'Tre Mann', 'first_name': 'Tre', 'last_name': 'Mann', 'is_active': True}
{'projection_id': '2275455', 'player_name': 'Tre Mann', 'player_team': 'BOS', 'stat_type': 'points', 'line_score': 12.5, 'start_time': '2024-04-12T19:40:00-04:00', 'player_id': 1630544, 'accuracy': 0.9090909090909091, 'precision': 0.75, 'recall': 0.75, 'f1': 0.75, 'prediction': 'OVER'}
PREDICTING Aaron Nesmith AT 12.5 points (37/460)
{'id': 1630174, 'full_name': 'Aaron Nesmith', 'first_name': 'Aaron', 'last_name': 'Nesmith', 'is_active': True}
{'projection_id': '2272871', 'player_name': 'Aaron Nesmith', 'player_team': 'CLE', 'stat_type': 'points', 'line_score': 1

FileNotFoundError: [Errno 2] No such file or directory: './data/prizepicks_predicted_props.json'